**SQL: Structured Query Language**

In [ ]:
SQL is designed to work with relational data. This really just means pieces of data that are related to each other.

asterix: This indicates that this is the primary key for the table. A primary key is a unique identifier for a table. That is, there can only be unique values for this column entry. 

The columns that are the primary key for one table can also appear on other tables. This is known as a foreign key aka the primary key from a different ("foreign") table.

Connecting to a database:
Connect to our database by importing sqlite3 and running the following cell in our notebook. You'll need a cursor object (cur) to fetch results. Cursor objects allow you to keep track of which result set is which since it's possible to run multiple queries before you're done fetching the results of the first.

import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

For strings, use """str""": triple quotes have the added functionality of being able to use multiple lines within the same string


Wrapping results into dataframes:
    
import pandas as pd
cur.execute("""SELECT * FROM employees LIMIT 5;""")
df = pd.DataFrame(cur.fetchall())
##To get column names
df.columns = [x[0] for x in cur.description]
df.head()

*Alternate to .fetchall* --> When you would only like the first result (or one result as is the case in the example above) you can use the sqlite3 method .fetchone()

In [ ]:
Where condition: the WHERE clause filters query results by some condition
    
    Example: cur.execute("""SELECT * FROM customers WHERE city = 'Boston' OR city = 'Madrid';""")

Order by and limit clauses:

Two additional keywords that you can use to refine your searches are the ORDER BY and LIMIT clauses. 
    -The order by clause allows you to sort the results by a particular feature. 
        -For example, you could sort by the customerName column if you wished to get results in alphabetical order. 
        -By default, ORDER BY is ascending. 
        -If you want the opposite, use the additional parameter DESC. 
        
    -Finally, the limit clause is typically the last argument in a SQL query and simply limits the output to a set number of results.

In [ ]:
Between clause:
    
Ex: SELECT column_name(s) FROM table_name WHERE column_name BETWEEN value1 AND value2;

In [ ]:
To find null:
    
Ex: SELECT * FROM cats WHERE Name IS null;

COUNT (SQL aggregate function)

For now, we'll just focus on COUNT, which counts the number of records that meet a certain condition. Here's a standard SQL query using COUNT:

SELECT COUNT([column name]) FROM [table name] WHERE [column name] = [value]
Let's try it out and count the number of cats who have an owner_id of 1:

SELECT COUNT(owner_id) FROM cats WHERE owner_id = 1;

In [ ]:
Group by

Like its name suggests, it groups your results by a given column.

Ex: SELECT breed, COUNT(breed) FROM cats GROUP BY breed;

SQLite allows us to explicitly state the tableName.columnName you want to select. This is particularly useful when you want data from two different tables.